In [1]:
import random

random.seed(321)

In [2]:
payment_table = {"Reward": 6,
                 "Sucker": 0,
                 "Temptation": 10,
                 "Punishment": 2}

In [3]:
import math

In [4]:
class Agent:
    def __init__(self, wealth, name):
        self.history = {}
        self.initial_wealth = wealth
        self.current_wealth = self.initial_wealth
        self.decision = None
        self.name = name
        self.wealth_history = [self.initial_wealth]

    def make_decision(self, cost2play):
        next_game_selection = []
        for key, value in payment_table.items():
            if self.current_wealth+value-cost2play > 0:
                next_game_selection.append(key)

        #print('agent name:', self.name)
        #print("current wealth:", self.current_wealth)
        #print("selection:", next_game_selection)
        if len(next_game_selection) >= 4:
            #All outcomes would allow survival
            #print(">= 4")
            return "C" if random.random() < 0.5 else "D"
        elif len(next_game_selection) == 3:
            #This means that Temptation, Reward, and Punishment would allow survival. So defection guarantees survival.
            #print("==3")
            return "D"
        elif len(next_game_selection) == 2:
            #This means that Temptation and Reward would allow survival
            #print("==2")
            return "D"
        elif len(next_game_selection) == 1:
            #This means the only option is Temptation, which is achieved only through defection
            #print("==1")
            return "D"
        else:
            #print("==0")
            return "C" if random.random() < 0.5 else "D"    
          
            
        '''
        #How many turns could you survive playing "C" in the worst case scenario
        
        if cost2play-payment_table["Sucker"] <= 0:
            coop_risk = 0
        else:
            coop_risk = 1/(self.current_wealth/(cost2play-payment_table["Sucker"]))
        
        #How many turns could you survive playing "D" in the worst case scenario
        if (cost2play-payment_table["Punishment"] <= 0):
            defect_risk = 0     
        else:
            defect_risk = 1/(self.current_wealth/(cost2play-payment_table["Punishment"]))

        if coop_risk < random.random():
            decision = "C"
        else:
            decision = "D"
        
        return decision
        '''

    def receive_payment(self, outcome):
        self.current_wealth += payment_table[outcome]
    
    def update_history(self, opponent, outcome):
        if opponent in self.history:
            self.history[opponent].append(outcome)
        else:
            self.history[opponent] = [outcome]

    def update_wealth_history(self):
        self.wealth_history.append(self.current_wealth)

    def pay2play(self, cost):
        self.current_wealth = self.current_wealth-cost

In [144]:
len([])

0

In [5]:
class PD:
    def __init__(self, cost2play):
        self.cost2play = cost2play

    def play(self, agent_1, agent_2):
        #print('Agent 1', agent_1.name)
        #print('Agent 2', agent_2.name)
        decision_1 = agent_1.make_decision(self.cost2play)
        decision_2 = agent_2.make_decision(self.cost2play)
        outcome_1, outcome_2 = self.calc_outcome(decision_1, decision_2)
        #print('Decision 1', decision_1)
        #print('Decision 2', decision_2)
        #print('########################')
        agent_1.receive_payment(outcome_1)
        agent_2.receive_payment(outcome_2)
        agent_1.update_history(agent_2.name, outcome_1)
        agent_2.update_history(agent_1.name, outcome_2)
        agent_1.pay2play(self.cost2play)
        agent_2.pay2play(self.cost2play)
        agent_1.update_wealth_history()
        agent_2.update_wealth_history()
        return outcome_1

    def calc_outcome(self, decision_1, decision_2):
        if decision_1+decision_2 == "CC":
            return "Reward", "Reward"
        elif decision_1+decision_2 == "DD":
            return "Punishment", "Punishment"
        elif decision_1+decision_2 == "CD":
            return "Sucker", "Temptation"
        elif decision_1+decision_2 == "DC":
            return "Temptation", "Sucker"

In [6]:
agent1 = Agent(3, "agent1")
agent2 = Agent(2, "agent2")
agent3 = Agent(2, "agent3")
agent4 = Agent(30, "agent4")

game = PD(2)

for x in range(10):
    if agent1.current_wealth <= 0 and agent2.current_wealth <= 0:
        game.play(agent1, agent2)
    if agent1.current_wealth <= 0 and agent3.current_wealth <= 0:   
        game.play(agent1, agent3)
    if agent1.current_wealth <= 0 and agent4.current_wealth <= 0:
        game.play(agent1, agent4)
    if agent2.current_wealth <= 0 and agent3.current_wealth <= 0:
        game.play(agent2, agent3)
    if agent2.current_wealth <= 0 and agent4.current_wealth <= 0:
        game.play(agent2, agent4)
    if agent3.current_wealth <= 0 and agent4.current_wealth <= 0:
        game.play(agent3, agent4)
 

In [474]:
agent1.current_wealth

3

In [316]:
agent2.current_wealth

0

In [317]:
agent3.current_wealth

0

In [318]:
agent4.current_wealth

31

In [ ]:
##################################
#####      ANALYSIS   ############
##################################

In [7]:
#Games to test: 0,1,2,2.3,2.5,2.7,2.8,3,3
game = PD(2)

In [948]:
#Splits to test:
#Up ramp
#[100,200,300,400]

#Down ramp
#[400,300,200,100]

#V Shape
#[100,400,400,100]

#^ Shape
#[400,100,100,400]

#-_-_
#[400,100,400,100]

#_-_-
#[100,400,100,400]

#__--
#[100,100,400,400]

#--__
#[400,400,100,100]

#Extreme Up Ramp
#[10,100,390,500]

#Extreme Down Ramp
#[500,390,100,10]

#Low Only
#[1000,0,0,0]

#Mid_Low Only
#[0,1000,0,0]

#Mid_High Only
#[0,0,1000,0]

#High Only
#[0,0,0,1000]

In [8]:
histogram_dict={"Up Ramp":[100,200,300,400],
                "Down Ramp":[400,300,200,100],
                "Peak Shape":[100,400,400,100],
                "V Shape":[400,100,100,400],
                "-_-_ Shape":[400,100,400,100],
                "_-_- Shape":[100,400,100,400],
                "__-- Shape":[100,100,400,400],
                "--__ Shape":[400,400,100,100],
                "Extreme Up Ramp":[10,100,390,500],
                "Extreme Down Ramp":[500,390,100,10],
                "Low Only":[1000,0,0,0],
                "Mid_Low Only":[0,1000,0,0],
                "Mid_High_Only":[0,0,1000,0],
                "High Only": [0,0,0,1000]
}

In [6]:
#split = [10,300,600,90]

#assert(sum(split)==1000)

In [9]:
def create_agents(split):
    agents = []
    for i in range(split[0]):
        agents.append(Agent(1,"low_"+str(i)))
    for i in range(split[1]):
        agents.append(Agent(2, "mid_low_"+str(i)))
    for i in range(split[2]):
        agents.append(Agent(6, "mid_high_"+str(i)))
    for i in range(split[3]):
        agents.append(Agent(10, "high_"+str(i)))
    return agents

In [10]:
import itertools

In [244]:
l = [1,2,3]
l[0:1]

[1]

In [11]:
def run_game(agents, our_game):
    agent_indices = list(range(len(agents)))
    random.shuffle(agent_indices)

    number_of_rounds = int(len(agents)/2*(len(agents)-1))
    outcomes = []
    game_count = 0
    for j in range(200):
        for i in range(number_of_rounds):
            top = agent_indices[:int(len(agent_indices)/2)]
            bottom = agent_indices[int(len(agent_indices)/2):]
            for idx, top_agent in enumerate(top):
                if agents[top_agent].current_wealth > 0 and agents[bottom[idx]].current_wealth > 0:
                    outcome = our_game.play(agents[top_agent], agents[bottom[idx]])
                    outcomes.append(outcome+"_round_"+str(i))
                    game_count += 1
            agent_indices = agent_indices[0:1]+agent_indices[-1:]+agent_indices[1:-1]
        
            
    '''
    pairs = list(itertools.combinations(list(range(len(agents))), 2))
    random.shuffle(pairs)
    game_count = 0
    outcomes = []
    for x in range(1):
        for pair in pairs:
            #print('Pair', pair)
            #print('Agent '+str(pair[0]), agents[pair[0]].current_wealth)
            #print('Agent '+str(pair[1]), agents[pair[1]].current_wealth)
            if agents[pair[0]].current_wealth > 0 and agents[pair[1]].current_wealth > 0:
                outcome = our_game.play(agents[pair[0]], agents[pair[1]])
                #print('After game')
                #print('Agent '+str(pair[0]), agents[pair[0]].current_wealth)
                #print('Agent '+str(pair[1]), agents[pair[1]].current_wealth)
                #print('')
                outcomes.append(outcome)
                game_count += 1
    '''
    return game_count, outcomes
    

In [12]:
def calc_avg_for_each(agents,splits):
    low_wealth_avg = 0
    mid_low_wealth_avg = 0
    mid_high_wealth_avg = 0
    high_wealth_avg = 0
    for agent in agents[:splits[0]]:
        low_wealth_avg += agent.current_wealth
    if splits[0]:
        low_wealth_avg = low_wealth_avg/splits[0]
    for agent in agents[splits[0]:sum(splits[:2])]:
        mid_low_wealth_avg += agent.current_wealth
    if (sum(splits[:2])-sum(splits[:1])):
        mid_low_wealth_avg = mid_low_wealth_avg/(sum(splits[:2])-sum(splits[:1]))
    for agent in agents[sum(splits[:2]):sum(splits[:3])]:
        mid_high_wealth_avg += agent.current_wealth
    if (sum(splits[:3])-sum(splits[:2])):
        mid_high_wealth_avg = mid_high_wealth_avg/(sum(splits[:3])-sum(splits[:2]))
    for agent in agents[sum(splits[:3]):]:
        high_wealth_avg += agent.current_wealth
    if (sum(splits)-sum(splits[:3])):
        high_wealth_avg = high_wealth_avg/(sum(splits)-sum(splits[:3]))

    return low_wealth_avg,mid_low_wealth_avg,mid_high_wealth_avg,high_wealth_avg

In [13]:
def remove_zeroed_agents(agents, splits):
    temp_splits = splits.copy()
    non_zero_agents = []
    non_zero_splits = []
    for agent in agents:
        if agent.current_wealth > 0:
            non_zero_agents.append(agent)
        else:
            if agent.name.startswith("low"):
                temp_splits[0] = temp_splits[0]-1
            elif agent.name.startswith("mid_low"):
                temp_splits[1] = temp_splits[1]-1
            elif agent.name.startswith("mid_high"):
                temp_splits[2] = temp_splits[2]-1
            elif agent.name.startswith("high"):
                temp_splits[3] = temp_splits[3]-1
    non_zero_splits = temp_splits
    return non_zero_agents, non_zero_splits

In [12]:
def count_zeroed_agents(agents):
    low_zeroed_count = 0
    mid_low_zeroed_count = 0
    mid_high_zeroed_count = 0
    high_zeroed_count = 0
    for agent in agents:
        if agent.current_wealth <= 0:
            if agent.name.startswith("low"):
                low_zeroed_count += 1
            elif agent.name.startswith("mid_low"):
                mid_low_zeroed_count += 1
            elif agent.name.startswith("mid_high"):
                mid_high_zeroed_count += 1
            elif agent.name.startswith("high"):
                high_zeroed_count += 1
    return low_zeroed_count, mid_low_zeroed_count, mid_high_zeroed_count, high_zeroed_count 
    

In [14]:
def count_coops(outcomes):
    count = 0
    strict_count = 0
    for outcome in outcomes:
        if not outcome.startswith("Punishment"):
            count += 1
        if outcome.startswith("Reward"):
            strict_count += 1
    return count, strict_count

In [15]:
def count_defections(outcomes):
    count = 0
    strict_count = 0
    for outcome in outcomes:
        if not outcome.startswith("Reward"):
            count += 1
        if outcome.startswith("Punishment"):
            strict_count += 1
    return count, strict_count

In [16]:
def print_statistics(cost, experiment_name, game_agents, experiment_split, number_of_games_played, game_outcomes, write2file=True):   
    print("Experiment Name:", experiment_name)
    print("Cost 2 Play:", cost)
    temp_experiment_split=[]
    temp_experiment_split = experiment_split.copy()
    low_wealth_avg,mid_low_wealth_avg,mid_high_wealth_avg,high_wealth_avg = calc_avg_for_each(game_agents, experiment_split)
    print("Low Wealth Average:", low_wealth_avg)
    print("Mid Low Wealth Average:", mid_low_wealth_avg)
    print("Mid High Wealth Average:", mid_high_wealth_avg)
    print("High Wealth Average:", high_wealth_avg)
    print("")
    print("Difference between High and Low:", high_wealth_avg-low_wealth_avg)
    print("")
    print("")
    non_zero_game_agents, non_zero_experiment_splits = remove_zeroed_agents(game_agents, temp_experiment_split)
    low_wealth_avg_no_zeros,mid_low_wealth_avg_no_zeros,mid_high_wealth_avg_no_zeros,high_wealth_avg_no_zeros = calc_avg_for_each(non_zero_game_agents, non_zero_experiment_splits)
    print("DISREGARDING AGENTS THAT ZEROED")
    print("Low Wealth Average:", low_wealth_avg_no_zeros)
    print("Mid Low Wealth Average:", mid_low_wealth_avg_no_zeros)
    print("Mid High Wealth Average:", mid_high_wealth_avg_no_zeros)
    print("High Wealth Average:", high_wealth_avg_no_zeros)
    print("")
    print("Difference between High and Low:", high_wealth_avg_no_zeros-low_wealth_avg_no_zeros)
    print("")
    print("")
    print("ZEROED AGENT COUNTS")
    low_zeroed_count, mid_low_zeroed_count, mid_high_zeroed_count, high_zeroed_count = count_zeroed_agents(game_agents)
    print("Total zeroed agents:", low_zeroed_count + mid_low_zeroed_count + mid_high_zeroed_count + high_zeroed_count)
    print("")
    print("Low zeroed agents:", low_zeroed_count)
    if experiment_split[0]:
        print("Percent Low zeroed agents:", low_zeroed_count/experiment_split[0])
    else:
        print("Percent Low zeroed agents:", 0)
    print("Mid low zeroed agents:", mid_low_zeroed_count)
    if experiment_split[1]:
        print("Percent Mid low zeroed agents:", mid_low_zeroed_count/experiment_split[1])
    else:
        print("Percent Mid low zeroed agents:", 0)
    print("Mid High zeroed agents:", mid_high_zeroed_count)
    if experiment_split[2]:
        print("Percent Mid high zeroed agents:", mid_high_zeroed_count/experiment_split[2])
    else:
       print("Percent Mid high zeroed agents:", 0) 
    print("High zeroed agents:", high_zeroed_count)
    if experiment_split[3]:
        print("Percent High zeroed agents:", mid_high_zeroed_count/experiment_split[3])
    else:
        print("Percent High zeroed agents:", 0)
    print("")
    print("")
    print("COOPERATION")
    num_coops, num_strict_coops = count_coops(game_outcomes)
    print("Number of Games that included Cooperation:", num_coops)
    print("Percentage of games that included a Cooperation:", num_coops/number_of_games_played)
    print("Number of Reward states:", num_strict_coops)
    print("Percentage of games with Reward state:", num_strict_coops/number_of_games_played)
    print("")
    print("DEFECTION") 
    num_defects, num_strict_defects = count_defections(game_outcomes)
    print("Number of Games that included Defection:", num_defects)
    print("Percentage of games that included a Defection:", num_defects/number_of_games_played)
    print("Number of Punishment states:", num_strict_defects)
    print("Percentage of games with Punishment state:", num_strict_defects/number_of_games_played)
    print("")

    if write2file:
        print("Writing to file:",experiment_name+"_"+str(cost)+".org") 
        f = open(experiment_name+"_"+str(cost)+".org", "a")
        f.write("Experiment Name: "+experiment_name+"\n")
        f.write("Cost 2 Play:"+str(cost)+"\n")
        f.write("Low Wealth Average: "+str(low_wealth_avg)+"\n")
        f.write("Mid Low Wealth Average: "+str(mid_low_wealth_avg)+"\n")
        f.write("Mid High Wealth Average: "+str(mid_high_wealth_avg)+"\n")
        f.write("High Wealth Average: "+str(high_wealth_avg)+"\n")
        f.write("\n")
        f.write("Difference between High and Low: "+str(high_wealth_avg-low_wealth_avg)+"\n")
        f.write("\n")
        f.write("\n")
        f.write("DISREGARDING AGENTS THAT ZEROED\n")
        f.write("Low Wealth Average: "+str(low_wealth_avg_no_zeros)+"\n")
        f.write("Mid Low Wealth Average: "+str(mid_low_wealth_avg_no_zeros)+"\n")
        f.write("Mid High Wealth Average: "+str(mid_high_wealth_avg_no_zeros)+"\n")
        f.write("High Wealth Average: "+str(high_wealth_avg_no_zeros)+"\n")
        f.write("\n")
        f.write("Difference between High and Low: "+str(high_wealth_avg_no_zeros-low_wealth_avg_no_zeros)+"\n")
        f.write("\n")
        f.write("\n")
        f.write("ZEROED AGENT COUNTS\n")
        f.write("Total zeroed agents: "+str(low_zeroed_count + mid_low_zeroed_count + mid_high_zeroed_count + high_zeroed_count)+"\n")
        f.write("\n")
        f.write("Low zeroed agents: "+str(low_zeroed_count)+"\n")
        if experiment_split[0]:
            f.write("Percent Low zeroed agents: "+str(low_zeroed_count/experiment_split[0])+"\n")
        else:
            f.write("Percent Low zeroed agents: 0\n")
        f.write("Mid low zeroed agents: "+str(mid_low_zeroed_count)+"\n")
        if experiment_split[1]:
            f.write("Percent Mid low zeroed agents: "+str(mid_low_zeroed_count/experiment_split[1])+"\n")
        else:
            f.write("Percent Mid low zeroed agents: 0\n")
        f.write("Mid High zeroed agents: "+str(mid_high_zeroed_count)+"\n")
        if experiment_split[2]:
            f.write("Percent Mid High zeroed agents: "+str(mid_high_zeroed_count/experiment_split[2])+"\n")
        else:
            f.write("Percent Mid High zeroed agents: 0\n")
        f.write("High zeroed agents: "+str(high_zeroed_count)+"\n")
        if experiment_split[3]:
            f.write("Percent High zeroed agents: "+str(high_zeroed_count/experiment_split[3])+"\n")
        else:
            f.write("Percent High zeroed agents: 0\n")
        f.write("\n")
        f.write("\n")
        f.write("COOPERATION\n")
        f.write("Number of Games that included Cooperation: "+str(num_coops)+"\n")
        f.write("Percentage of games that included a Cooperation "+str(num_coops/number_of_games_played)+"\n")
        f.write("Number of Reward states: "+str(num_strict_coops)+"\n")
        f.write("Percentage of games with Reward state: "+str(num_strict_coops/number_of_games_played)+"\n")
        f.write("\n")
        f.write("DEFECTION\n") 
        f.write("Number of Games that included Defection: "+str(num_defects)+"\n")
        f.write("Percentage of games that included a Defection: "+str(num_defects/number_of_games_played)+"\n")
        f.write("Number of Punishment states: "+str(num_strict_defects)+"\n")
        f.write("Percentage of games with Punishment state: "+str(num_strict_defects/number_of_games_played)+"\n")
        f.write("\n")

In [17]:
def run_analysis(histogram_dict, cost2play_list):
    for cost in cost2play_list:
        game = PD(cost)

        for experiment_name, experiment_split in histogram_dict.items():
            game_agents = create_agents(experiment_split)
            number_of_games_played, game_outcomes = run_game(game_agents, game)

            print_statistics(cost,experiment_name,game_agents, experiment_split, number_of_games_played, game_outcomes, write2file=False)
            #print(game_outcomes)
            #print(game_agents[0].name)
            #print(game_agents[0].history)
            #print(game_agents[0].wealth_history)

In [287]:
run_analysis({'Custom': [100,2,6,0]}, [2])

Experiment Name: Custom
Cost 2 Play: 2
Low Wealth Average: 2888645.94
Mid Low Wealth Average: 2890713.0
Mid High Wealth Average: 2889294.0
High Wealth Average: 0

Difference between High and Low: -2888645.94


DISREGARDING AGENTS THAT ZEROED
Low Wealth Average: 2888645.94
Mid Low Wealth Average: 2890713.0
Mid High Wealth Average: 2889294.0
High Wealth Average: 0

Difference between High and Low: -2888645.94


ZEROED AGENT COUNTS
Total zeroed agents: 0

Low zeroed agents: 0
Percent Low zeroed agents: 0.0
Mid low zeroed agents: 0
Percent Mid low zeroed agents: 0.0
Mid High zeroed agents: 0
Percent Mid high zeroed agents: 0.0
High zeroed agents: 0
Percent High zeroed agents: 0


COOPERATION
Number of Games that included Cooperation: 46796442
Percentage of games that included a Cooperation: 0.7499141379177724
Number of Reward states: 15601496
Percentage of games with Reward state: 0.25001435842211195

DEFECTION
Number of Games that included Defection: 46800904
Percentage of games that incl

In [150]:
################################################

In [18]:
num_of_games_that_contained_a_defection = 0
for outcome in outcomes:
    if not outcome == 'Reward':
        num_of_games_that_contained_a_defection += 1

num_of_games_that_contained_cooperation = 0
for outcome in outcomes:
    if not outcome == 'Punishment':
        num_of_games_that_contained_cooperation += 1


NameError: name 'outcomes' is not defined

In [930]:
num_of_games_that_contained_a_defection/game_count

0.000684735321086645

In [931]:
num_of_games_that_contained_cooperation/game_count

0.9999851293699062

In [934]:
non_zero_agents, non_zero_splits = remove_zeroed_agents(agents, split)

In [935]:
non_zero_splits

[4, 300, 600, 90]

In [936]:
split

[10, 300, 600, 90]

In [937]:
low_wealth_avg,mid_low_wealth_avg,mid_high_wealth_avg,high_wealth_avg = calc_avg_for_each(agents, split)

In [938]:
low_wealth_avg,mid_low_wealth_avg,mid_high_wealth_avg,high_wealth_avg

(26013.13999995594, 19528.551333300235, 16.0, 25.0)

In [939]:
low_wealth_avg_no_zeros,mid_low_wealth_avg_no_zeros,mid_high_wealth_avg_no_zeros,high_wealth_avg_no_zeros = calc_avg_for_each(non_zero_agents, non_zero_splits)

In [940]:
low_wealth_avg_no_zeros,mid_low_wealth_avg_no_zeros,mid_high_wealth_avg_no_zeros,high_wealth_avg_no_zeros

(65033.92499988985, 19528.551333300235, 16.0, 25.0)

In [941]:
#Zeroed agents gap
high_wealth_avg-low_wealth_avg

-25988.13999995594

In [942]:
#Non-zeroed agents gap
high_wealth_avg_no_zeros-low_wealth_avg_no_zeros

-65008.92499988985

In [ ]:
################################

In [606]:
mid_low_wealth_avg = 0
for agent in agents[split[0]:sum(split[:2])]:
    mid_low_wealth_avg += agent.current_wealth
mid_low_wealth_avg = mid_low_wealth_avg/(sum(split[:2])-sum(split[:1]))
print(mid_low_wealth_avg)

95987.3


In [601]:
mid_high_wealth_avg = 0
for agent in agents[sum(split[:2]):sum(split[:3])]:
    mid_high_wealth_avg += agent.current_wealth
mid_high_wealth_avg = mid_high_wealth_avg/(sum(split[:3])-sum(split[:2]))
print(mid_high_wealth_avg)

96001.76


In [605]:
high_wealth_avg = 0
for agent in agents[sum(split[:3]):]:
    high_wealth_avg += agent.current_wealth
high_wealth_avg = high_wealth_avg/(sum(split)-sum(split[:3]))
print(high_wealth_avg)

96012.2


In [378]:
19628.0-18917.7

710.2999999999993

In [379]:
25-4

21

In [ ]:
#For a 30,59,10,1 split
#The difference between low and high is 710.2999999999993
#And the initial difference is 21

In [388]:
19421.64-17823.32

1598.3199999999997

In [ ]:
#For a 25,25,25,25 split
#The difference between low and high is 1598
#And the initial difference is 21

In [397]:
19815-19781.0

34.0

In [ ]:
#For a 1,10,59,30 split
#The difference between low and high is 34

In [405]:
19014.163265306124-17431.775510204083

1582.3877551020414

In [ ]:
#For a 49,1,1,49 split
#The difference between low and high is 1582.3877551020414

In [413]:
19827.0-19739.0

88.0

In [ ]:
#Fpr a 1,49,49,1 split
#The difference between low and high is 88

In [421]:
19814.208333333332-19782.5

31.70833333333212

In [ ]:
#For a 2,2,48,48 split
#The difference between low and high is 31.70833333333212

In [429]:
19811.521276595744-19791.5

20.021276595743984

In [ ]:
#For a 2,2,2,94 split
#The difference between low and high is 20.021276595743984

In [436]:
18621.5-17399.32978723404

1222.1702127659591

In [ ]:
#For a 94,2,2,2 split
#The difference between low and high is 1222.1702127659591

In [ ]:
#########################################
#########TEST 2##########################
#########################################

In [ ]:
def run_game(agents, game):
    agent_indices = list(range(len(agents)))
    random.shuffle(agent_indices)

    